# 0 - Load data

This notebook helps to load data from gzipped JSON lines files into MongoDB.

In [1]:
import gzip
import json
import pymongo
from tqdm.notebook import tqdm

In [2]:
mongo_client = pymongo.MongoClient(
    host="127.0.0.1",
    port=27017,
)
mongo_repositories = mongo_client["gha-scraper"]["repositories"]
mongo_runs         = mongo_client["gha-scraper"]["runs"]

### Restore JSONs to MongoDB

In [ ]:
with gzip.open("repositories.json.gz", "rt") as fd:
    for line in tqdm(fd):
        mongo_repositories.insert_one(json.loads(fd.read()))

with gzip.open("runs.json.gz", "rt") as fd:
    for line in tqdm(fd):
        mongo_runs.insert_one(json.loads(fd.read()))

In [ ]:
mongo_repositories.create_index("selected")
mongo_runs.create_index("repository_name")
mongo_runs.create_index("log_insights")
mongo_runs.create_index("logs_archive")
mongo_runs.create_index(["repository_name", "workflow_path"])

### Backup MongoDB to JSONs

In [3]:
with gzip.open("repositories.json.gz", "wt") as fd:
    for repository in tqdm(mongo_repositories.find()):
        fd.write(json.dumps(repository, default=str) + "\n")

with gzip.open("runs.json.gz", "wt") as fd:
    for run in tqdm(mongo_runs.find()):
        fd.write(json.dumps(run, default=str) + "\n")

0it [00:00, ?it/s]

0it [00:00, ?it/s]